<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_OPENMODELDEMO_AAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agentic AI refers to AI systems designed to operate autonomously, making decisions and taking actions to achieve specific goals with limited or no human supervision. Unlike traditional AI, which often operates within predefined rules and requires human input, agentic AI exhibits autonomy, goal-driven behavior, and adaptability. These systems utilize AI agents, which are machine learning models that mimic human decision-making to solve problems in real-time.

In [ ]:
!pip install mistralai -q
!pip install colab-env -q
import colab_env

In [6]:
import os
import colab_env

from mistralai import Mistral
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()


model_list = client.models.list()

# Print all model IDs
#for model_info in model_list.data:
#    print(model_info.id)

model = "open-mixtral-8x22b"

messages = [{"role": "user", "content": "What is the best Cuban poet?"}]

chat_response = client.chat.complete(
    model=model,  # Pass the model ID string
    messages=messages,
)
print('\n')
print(chat_response.choices[0].message.content)


print('\n\n')

historical_context_agent = client.beta.agents.create(
    model = "open-mixtral-8x22b",
    description="Agent for providing historical context and the intellectual lineage of scientific discoveries, including the contributions of figures like Newton, Galileo, Einstein, and Hinton.",
    name="historical-context-agent",
    tools=[
        # Removed {"type": "web_search"} as magistral-medium-latest does not support built-in connectors
        {
            "type": "function",
            "function": {
                "name": "get_scientist_contributions",
                "description": "Retrieve key scientific contributions of a historical figure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "scientist_name": {"type": "string", "description": "Name of the scientist (e.g., 'Isaac Newton', 'Albert Einstein')."},
                        "field_of_study": {"type": "string", "description": "Optional: Specific field of study to narrow down contributions."}
                    },
                    "required": ["scientist_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_intellectual_synergy",
                "description": "Analyze the intellectual synergy or influence between different historical scientific figures.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "figure_A": {"type": "string", "description": "First scientific figure."},
                        "figure_B": {"type": "string", "description": "Second scientific figure."},
                        "common_theme": {"type": "string", "description": "Optional: Common theme or area of influence (e.g., 'relativity', 'mechanics', 'AI')."}
                    },
                    "required": ["figure_A", "figure_B"]
                }
            }
        }
    ]
)
print(f"Historical Context Agent '{historical_context_agent.name}' created with ID: {historical_context_agent.id}")



It's subjective to determine the "best" Cuban poet, as poetry appreciation can vary greatly from person to person. However, one of the most renowned and celebrated Cuban poets is José Martí. He is often referred to as the "Apostle of Cuban Independence" because of his significant contributions to the Cuban War of Independence, both militarily and as a writer. His poetry and essays stirred strong feelings of patriotism in the Cuban people.

Another notable Cuban poet is Nicolás Guillén, who is recognized as the national poet of Cuba. His poetry often incorporated elements of Afro-Cuban culture and rhythm. Other notable Cuban poets include Dulce María Loynaz, Nancy Morejón, and Reina María Rodríguez.



Historical Context Agent 'historical-context-agent' created with ID: ag_0197d810740b7096aec35be72b5feaa3


## AGENTINC AI

In [14]:
import math
from datetime import datetime, timedelta
import random
import os
import json

# Import Mistral API client as per user's reference
from mistralai import Mistral


# --- Mistral API Setup ---
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)
MISTRAL_MODEL = "open-mixtral-8x22b" # Or another suitable model like "mistral-large-latest"

# --- Simulated Knowledge Bases ---
AIRCRAFT_DATA = {
    'BOEING 737': {'speed': 450, 'fuel_consumption_rate': 1500, 'range': 3000, 'optimal_altitude': 35000},  # mph, lbs/hr, miles, feet
    'AIRBUS A320': {'speed': 470, 'fuel_consumption_rate': 1600, 'range': 3200, 'optimal_altitude': 37000},
    'CESSNA 172': {'speed': 120, 'fuel_consumption_rate': 8, 'range': 600, 'optimal_altitude': 8000},
    'BOEING 777': {'speed': 550, 'fuel_consumption_rate': 3000, 'range': 8000, 'optimal_altitude': 40000}, # Added Boeing 777
}

AIRPORT_DATA = {
    'JFK': {'name': 'John F. Kennedy International Airport', 'lat': 40.6413, 'lon': -73.7781, 'runways': 'Long, Multiple', 'services': 'Full'},
    'LAX': {'name': 'Los Angeles International Airport', 'lat': 33.9416, 'lon': -118.4085, 'runways': 'Long, Multiple', 'services': 'Full'},
    'ORD': {'name': 'O\'Hare International Airport', 'lat': 41.9742, 'lon': -87.9073, 'runways': 'Long, Multiple', 'services': 'Full'},
    'MIA': {'name': 'Miami International Airport', 'lat': 25.7959, 'lon': -80.2870, 'runways': 'Long, Multiple', 'services': 'Full'},
    'YYZ': {'name': 'Toronto Pearson International Airport', 'lat': 43.6777, 'lon': -79.6248, 'runways': 'Long, Multiple', 'services': 'Full'},
    'YUL': {'name': 'Montréal-Trudeau International Airport', 'lat': 45.4706, 'lon': -73.7408, 'runways': 'Long, Multiple', 'services': 'Full'},
    'LHR': {'name': 'London Heathrow Airport', 'lat': 51.4700, 'lon': -0.4543, 'runways': 'Long, Multiple', 'services': 'Full'},
    'CDG': {'name': 'Paris Charles de Gaulle Airport', 'lat': 49.0097, 'lon': 2.5479, 'runways': 'Long, Multiple', 'services': 'Full'},
    'ATL': {'name': 'Hartsfield-Jackson Atlanta International Airport', 'lat': 33.6407, 'lon': -84.4277, 'runways': 'Long, Multiple', 'services': 'Full'},
    'DFW': {'name': 'Dallas/Fort Worth International Airport', 'lat': 32.8998, 'lon': -97.0403, 'runways': 'Long, Multiple', 'services': 'Full'},
    'EWR': {'name': 'Newark Liberty International Airport', 'lat': 40.6895, 'lon': -74.1745, 'runways': 'Long, Multiple', 'services': 'Full'}, # Added for alternate
    'BOS': {'name': 'Boston Logan International Airport', 'lat': 42.3643, 'lon': -71.0052, 'runways': 'Long, Multiple', 'services': 'Full'}, # Added for alternate
    'PVG': {'name': 'Shanghai Pudong International Airport', 'lat': 31.1434, 'lon': 121.8053, 'runways': 'Long, Multiple', 'services': 'Full'}, # Added Shanghai
}

# --- Tool Definitions (Python functions that the orchestrator can call) ---

def calculate_distance_tool(lat1, lon1, lat2, lon2):
    """
    Calculates the great-circle distance between two geographical points (airports) in miles.
    Args:
        lat1 (float): Latitude of the first point.
        lon1 (float): Longitude of the first point.
        lat2 (float): Latitude of the second point.
        lon2 (float): Longitude of the second point.
    Returns:
        float: Distance in miles.
    """
    R = 3958.8  # Radius of Earth in miles
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = (math.sin(dLat / 2) * math.sin(dLat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dLon / 2) * math.sin(dLon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));
    distance = R * c
    return distance

def get_simulated_weather_tool(airport_code, severity='normal'):
    """
    Simulates weather conditions for a given airport.
    Args:
        airport_code (str): ICAO code of the airport.
        severity (str): 'normal', 'moderate', or 'bad' to simulate different conditions.
    Returns:
        str: A string describing the simulated weather.
    """
    base_weather = {
        'JFK': 'Clear skies, 15°C, winds 10 knots from NW',
        'LAX': 'Sunny, 22°C, winds 5 knots from SW',
        'ORD': 'Partly cloudy, 10°C, winds 12 knots from N',
        'MIA': 'Humid, 28°C, winds 8 knots from SE',
        'YYZ': 'Cloudy, 8°C, winds 15 knots from E',
        'YUL': 'Overcast, 5°C, winds 18 knots from NE',
        'LHR': 'Foggy, 7°C, winds 7 knots from W',
        'CDG': 'Rainy, 12°C, winds 20 knots from SW',
        'ATL': 'Scattered clouds, 20°C, winds 7 knots from S',
        'DFW': 'Sunny, 25°C, winds 10 knots from SE',
        'EWR': 'Partly cloudy, 18°C, winds 10 knots from W',
        'BOS': 'Clear skies, 12°C, winds 8 knots from SW',
        'PVG': 'Partly cloudy, 20°C, winds 10 knots from SE', # Simulated weather for Shanghai
    }

    weather = base_weather.get(airport_code.upper(), 'Unknown weather conditions.')

    if severity == 'bad':
        weather += ' - **SEVERE THUNDERSTORMS EXPECTED! Visibility low.**'
    elif severity == 'moderate':
        weather += ' - Moderate turbulence reported. Light rain.'
    return weather

# --- Agent Definitions (Conceptual roles for LLM calls) ---
# These are not Mistral agent objects created via client.beta.agents.create
# but rather conceptual roles defined by system prompts for direct LLM calls.
AGENT_ROLES = {
    'user_input_agent': "You are a helpful assistant for gathering and clarifying initial flight requirements. Provide concise summaries.",
    'aircraft_performance_agent': "You are an expert in aircraft performance. Provide detailed data for specific aircraft models.",
    'airport_info_agent': "You are an airport information specialist. Provide comprehensive details about airports.",
    'route_calculation_agent': "You are a route planning expert. Calculate flight distances and provide conceptual route descriptions.",
    'origin_weather_agent': "You are an aviation meteorologist focused on departure airport weather. Analyze and summarize conditions and their impact.",
    'destination_weather_agent': "You are an aviation meteorologist focused on destination airport weather. Analyze and summarize conditions, potential landing hazards, and visibility.",
    'enroute_weather_agent': "You are an aviation meteorologist specializing in en-route weather. Analyze conditions along the flight path for turbulence, icing, and winds aloft.",
    'regulatory_compliance_agent': "You are an aviation regulatory expert. Identify key regulatory considerations, NOTAMs, and TFRs, and provide compliance advisories.",
    'fuel_load_agent': "You are a fuel and load planning specialist. Calculate precise fuel requirements and provide general considerations for aircraft weight and balance.",
    'contingency_planning_agent': "You are a contingency planning expert. Develop alternate airport strategies and advise on emergency procedures.",
    'final_synthesis_agent': "You are the lead flight planner. Synthesize all information from various specialized agents into a comprehensive and final flight plan. Use clear headings."
}

def get_llm_response_for_agent(agent_name, user_prompt, conversation_history=None):
    """
    Gets an LLM response, acting as a specific agent, using the Mistral API.
    Args:
        agent_name (str): The name of the conceptual agent (key in AGENT_ROLES).
        user_prompt (str): The user's query or information for this agent.
        conversation_history (list): Optional list of previous messages for context.
    Returns:
        The text content of the LLM's response.
    """
    system_message = AGENT_ROLES.get(agent_name, "You are a helpful AI assistant.")
    messages = [{"role": "system", "content": system_message}]

    if conversation_history:
        messages.extend(conversation_history)

    messages.append({"role": "user", "content": user_prompt})

    print(f"\n[Calling LLM as {agent_name} with prompt]: {user_prompt[:100]}...")
    try:
        chat_response = client.chat.complete(
            model=MISTRAL_MODEL,
            messages=messages,
        )
        return chat_response.choices[0].message.content
    except Exception as e:
        print(f"Error calling Mistral LLM for {agent_name}: {e}")
        return f"Error: {agent_name} could not provide a response due to an API error."


# --- Main Agentic Flight Planning Logic (Orchestration) ---

def plan_flight(departure_icao, destination_icao, aircraft_type, weather_severity='normal'):
    """
    Orchestrates the flight planning process using conceptual Mistral Agents and local tools.
    Args:
        departure_icao (str): ICAO code for the departure airport.
        destination_icao (str): ICAO code for the destination airport.
        aircraft_type (str): Type of aircraft.
        weather_severity (str): 'normal', 'moderate', or 'bad' for destination weather.
    Returns:
        A dictionary containing the flight plan details, or None if planning fails.
    """
    print(f"\n--- Initiating Multi-Agent Flight Plan for {departure_icao} to {destination_icao} with {aircraft_type} ---")

    # Initialize message histories for conceptual agents (not used for persistent Mistral agents)
    # These are just to pass context within a single `plan_flight` execution.
    agent_message_histories = {name: [] for name in AGENT_ROLES.keys()}

    # 1. User Input & Initial Validation (Orchestrator handles direct data access)
    dep_airport_data = AIRPORT_DATA.get(departure_icao.upper())
    dest_airport_data = AIRPORT_DATA.get(destination_icao.upper())
    aircraft_data_obj = AIRCRAFT_DATA.get(aircraft_type.upper())

    if not dep_airport_data or not dest_airport_data or not aircraft_data_obj:
        print(f"Error: Invalid airport codes or aircraft type provided.")
        return None

    print("\n[Orchestrator Step 1: Initial Data Gathering & Validation]")
    print(f"  Departure: {dep_airport_data['name']} ({departure_icao})")
    print(f"  Destination: {dest_airport_data['name']} ({destination_icao})")
    print(f"  Aircraft: {aircraft_type} (Speed: {aircraft_data_obj['speed']} mph, Fuel Rate: {aircraft_data_obj['fuel_consumption_rate']} lbs/hr)")

    # --- Step-by-step orchestration using conceptual agents ---
    flight_plan_summary = {}

    # Agent 1: User Input Agent (Simulated initial request processing)
    print("\n[Orchestrator Step 2: User Input Agent - Processing Request]")
    user_request_prompt = (
        f"The user wants to plan a flight from {departure_icao} to {destination_icao} using a {aircraft_type}. "
        "They need a comprehensive plan considering all factors. "
        f"The initial assumption for destination weather is '{weather_severity}'. "
        "Summarize the core request and identify initial key data points needed."
    )
    response_content = get_llm_response_for_agent('user_input_agent', user_request_prompt)
    flight_plan_summary['initial_request_processing'] = response_content
    print(f"  User Input Agent processed request: {flight_plan_summary['initial_request_processing']}")

    # Agent 2: Aircraft Performance Agent
    print("\n[Orchestrator Step 3: Aircraft Performance Agent - Retrieving Data]")
    aircraft_prompt = f"Provide detailed performance data for a {aircraft_type} including its speed, fuel consumption rate, range, and optimal cruising altitude. Use the following data: Speed={aircraft_data_obj['speed']} mph, Fuel Rate={aircraft_data_obj['fuel_consumption_rate']} lbs/hr, Range={aircraft_data_obj['range']} miles, Optimal Altitude={aircraft_data_obj['optimal_altitude']} feet. Format clearly."
    response_content = get_llm_response_for_agent('aircraft_performance_agent', aircraft_prompt)
    flight_plan_summary['aircraft_performance'] = response_content
    print(f"  Aircraft Performance Agent: {flight_plan_summary['aircraft_performance']}")

    # Agent 3: Airport Information Agent
    print("\n[Orchestrator Step 4: Airport Information Agent - Retrieving Data]")
    airport_prompt = f"Provide comprehensive details for {departure_icao} and {destination_icao} airports, including their full names, coordinates, available runways, and services. Use the following data: {departure_icao} - {AIRPORT_DATA.get(departure_icao.upper())}, {destination_icao} - {AIRPORT_DATA.get(destination_icao.upper())}. Format clearly."
    response_content = get_llm_response_for_agent('airport_info_agent', airport_prompt)
    flight_plan_summary['airport_info'] = response_content
    print(f"  Airport Information Agent: {flight_plan_summary['airport_info']}")

    # Agent 4: Route Calculation Agent (Orchestrator calls tool, then sends result to agent)
    print("\n[Orchestrator Step 5: Route Calculation Agent - Calculating Distance]")
    distance = calculate_distance_tool(dep_airport_data['lat'], dep_airport_data['lon'], dest_airport_data['lat'], dest_airport_data['lon'])
    flight_time_hours = distance / aircraft_data_obj['speed']

    route_prompt = (
        f"The direct flight distance between {dep_airport_data['name']} and {dest_airport_data['name']} is {distance:.2f} miles. "
        f"For a {aircraft_type} with a speed of {aircraft_data_obj['speed']} mph, the estimated flight time is {flight_time_hours:.2f} hours. "
        "Based on this, describe a conceptual route and any initial considerations for routing."
    )
    response_content = get_llm_response_for_agent('route_calculation_agent', route_prompt)
    flight_plan_summary['route_calculation'] = response_content
    print(f"  Route Calculation Agent: {flight_plan_summary['route_calculation']}")

    # Agent 5: Origin Weather Agent (Orchestrator calls tool, then sends result to agent)
    print("\n[Orchestrator Step 6: Origin Weather Agent - Getting Weather]")
    origin_weather_data = get_simulated_weather_tool(departure_icao, 'normal')
    origin_weather_prompt = f"Analyze the simulated weather conditions at {departure_icao}: '{origin_weather_data}'. What are the implications for takeoff and initial climb?"
    response_content = get_llm_response_for_agent('origin_weather_agent', origin_weather_prompt)
    flight_plan_summary['origin_weather'] = response_content
    print(f"  Origin Weather Agent: {flight_plan_summary['origin_weather']}")

    # Agent 6: Destination Weather Agent (Orchestrator calls tool, then sends result to agent)
    print("\n[Orchestrator Step 7: Destination Weather Agent - Getting Weather]")
    dest_weather_data = get_simulated_weather_tool(destination_icao, weather_severity)
    dest_weather_prompt = f"Analyze the simulated weather conditions at {destination_icao}: '{dest_weather_data}'. What are the implications for landing, visibility, and potential hazards?"
    response_content = get_llm_response_for_agent('destination_weather_agent', dest_weather_prompt)
    flight_plan_summary['destination_weather'] = response_content
    print(f"  Destination Weather Agent: {flight_plan_summary['destination_weather']}")

    # Agent 7: En-Route Weather Agent (Orchestrator calls tool, then sends result to agent)
    print("\n[Orchestrator Step 8: En-Route Weather Agent - Analyzing Route Weather]")
    # Simulate en-route weather for a random airport as a proxy
    enroute_proxy_airport = random.choice(list(AIRPORT_DATA.keys()))
    enroute_weather_data = get_simulated_weather_tool(enroute_proxy_airport, random.choice(['normal', 'moderate']))
    enroute_weather_prompt = f"Analyze potential en-route weather conditions for a flight from {departure_icao} to {destination_icao}, considering simulated conditions like '{enroute_weather_data}' (proxy from {enroute_proxy_airport}). What are the implications for turbulence, icing, and winds aloft, and recommended cruising altitude adjustments?"
    response_content = get_llm_response_for_agent('enroute_weather_agent', enroute_weather_prompt)
    flight_plan_summary['enroute_weather'] = response_content
    print(f"  En-Route Weather Agent: {flight_plan_summary['enroute_weather']}")

    # Agent 8: Regulatory Compliance Agent
    print("\n[Orchestrator Step 9: Regulatory Compliance Agent - Checking Regulations]")
    regulatory_prompt = f"For a flight from {departure_icao} to {destination_icao} with a {aircraft_type}, what key general regulatory considerations (e.g., FAA, ICAO), NOTAMs, or TFRs should be checked? Provide a general overview of compliance requirements."
    response_content = get_llm_response_for_agent('regulatory_compliance_agent', regulatory_prompt)
    flight_plan_summary['regulatory_compliance'] = response_content
    print(f"  Regulatory Compliance Agent: {flight_plan_summary['regulatory_compliance']}")

    # Agent 9: Fuel & Load Agent
    print("\n[Orchestrator Step 10: Fuel & Load Agent - Calculating Requirements]")
    fuel_required_calculated = flight_time_hours * aircraft_data_obj['fuel_consumption_rate']
    fuel_load_prompt = (
        f"Calculate the estimated fuel required for a {aircraft_type} flying {distance:.2f} miles, "
        f"with an estimated flight time of {flight_time_hours:.2f} hours and a fuel consumption rate of {aircraft_data_obj['fuel_consumption_rate']} lbs/hr. "
        f"The calculated fuel is {fuel_required_calculated:.2f} lbs. "
        "Also, provide general considerations for aircraft weight and balance based on this information."
    )
    response_content = get_llm_response_for_agent('fuel_load_agent', fuel_load_prompt)
    flight_plan_summary['fuel_load'] = response_content
    print(f"  Fuel & Load Agent: {flight_plan_summary['fuel_load']}")

    # Agent 10: Contingency Planning Agent (Orchestrator calls tool, then sends result to agent)
    print("\n[Orchestrator Step 11: Contingency Planning Agent - Developing Alternates]")
    # Check weather for a potential alternate airport (e.g., EWR for JFK/LAX flight)
    # For YUL to PVG, let's pick a plausible alternate for PVG, e.g., RJAA (Narita) or RKSI (Incheon)
    # Since we only have a few airports, let's use a nearby major airport from our list as a proxy
    alternate_airport_code = 'LHR' # Using LHR as a far-east proxy for an alternate
    alternate_weather_data = get_simulated_weather_tool(alternate_airport_code, 'normal')

    contingency_prompt = (
        f"Given the destination weather at {destination_icao} is '{weather_severity}' ({flight_plan_summary['destination_weather']}), "
        f"and considering a {aircraft_type} (range: {aircraft_data_obj['range']} miles), "
        f"and a potential alternate airport {alternate_airport_code} with weather '{alternate_weather_data}', "
        "suggest suitable alternate airports and provide advice on general contingency planning for this flight."
    )
    response_content = get_llm_response_for_agent('contingency_planning_agent', contingency_prompt)
    flight_plan_summary['contingency_planning'] = response_content
    print(f"  Contingency Planning Agent: {flight_plan_summary['contingency_planning']}")


    # --- Final Synthesis by the Orchestrator (using a dedicated LLM call for synthesis) ---
    print("\n[Orchestrator Step 12: Final Synthesis of Flight Plan]")
    final_synthesis_prompt = (
        "Synthesize the following information provided by various specialized agents into a comprehensive and final flight plan. "
        "Ensure it's well-structured with clear headings and covers all critical aspects for a flight from "
        f"{departure_icao} to {destination_icao} with a {aircraft_type}. "
        "Include estimated flight time and fuel, and a summary of all agent findings.\n\n"
        f"**Initial Request Processing:** {flight_plan_summary.get('initial_request_processing', 'N/A')}\n\n"
        f"**Aircraft Performance:** {flight_plan_summary.get('aircraft_performance', 'N/A')}\n\n"
        f"**Airport Information:** {flight_plan_summary.get('airport_info', 'N/A')}\n\n"
        f"**Route Calculation:** {flight_plan_summary.get('route_calculation', 'N/A')}\n\n"
        f"**Origin Weather:** {flight_plan_summary.get('origin_weather', 'N/A')}\n\n"
        f"**Destination Weather:** {flight_plan_summary.get('destination_weather', 'N/A')}\n\n"
        f"**En-Route Weather:** {flight_plan_summary.get('enroute_weather', 'N/A')}\n\n"
        f"**Regulatory Compliance:** {flight_plan_summary.get('regulatory_compliance', 'N/A')}\n\n"
        f"**Fuel & Load Calculation:** {flight_plan_summary.get('fuel_load', 'N/A')}\n\n"
        f"**Contingency Planning:** {flight_plan_summary.get('contingency_planning', 'N/A')}\n\n"
        f"**Calculated Direct Distance:** {distance:.2f} miles\n"
        f"**Estimated Flight Time:** {round(flight_time_hours * 60)} minutes\n"
        f"**Estimated Fuel Required:** {fuel_required_calculated:.2f} lbs\n"
        f"**Estimated Arrival Time:** {(datetime.now() + timedelta(minutes=round(flight_time_hours * 60))).strftime('%H:%M %Z')}\n"
    )
    final_llm_plan_details = get_llm_response_for_agent('final_synthesis_agent', final_synthesis_prompt)


    flight_plan = {
        'departure': dep_airport_data['name'],
        'destination': dest_airport_data['name'],
        'aircraft': aircraft_type,
        'distance': f"{distance:.2f}",
        'flight_time_minutes': round(flight_time_hours * 60),
        'fuel_required': f"{fuel_required_calculated:.2f}",
        'origin_weather': flight_plan_summary.get('origin_weather', 'N/A'),
        'destination_weather': flight_plan_summary.get('destination_weather', 'N/A'),
        'llm_plan_details': final_llm_plan_details,
        'estimated_arrival_time': (datetime.now() + timedelta(minutes=round(flight_time_hours * 60))).strftime('%H:%M %Z')
    }
    return flight_plan

# --- Execution Block (Fully Automatic) ---
if __name__ == "__main__":
    # Define fixed flight details for automatic execution
    departure_icao = "YUL" # Changed to Montreal
    destination_icao = "PVG" # Changed to Shanghai
    aircraft_type = "BOEING 777" # Changed to Boeing 777

    print(f"Automatically planning flight from {departure_icao} to {destination_icao} with {aircraft_type} using 10 conceptual agents...")

    current_flight_plan = plan_flight(departure_icao, destination_icao, aircraft_type)

    if current_flight_plan:
        print("\n--- FINAL COMPREHENSIVE FLIGHT PLAN SUMMARY ---")
        print(current_flight_plan['llm_plan_details']) # This now contains the full synthesized plan

        # Automatically simulate a weather change and re-plan
        new_severity = random.choice(['moderate', 'bad'])
        print(f"\n[Orchestrator Step: Feedback Loop - Automatically Simulating '{new_severity}' weather change at destination and re-planning]")
        re_planned_flight = plan_flight(departure_icao, destination_icao, aircraft_type, weather_severity=new_severity)
        if re_planned_flight:
            print("\n--- RE-PLANNED COMPREHENSIVE FLIGHT PLAN SUMMARY (Due to Weather Change) ---")
            print(re_planned_flight['llm_plan_details'])
        else:
            print("Automatic re-planning failed.")
    else:
        print("Initial flight planning failed.")


Automatically planning flight from YUL to PVG with BOEING 777 using 10 conceptual agents...

--- Initiating Multi-Agent Flight Plan for YUL to PVG with BOEING 777 ---

[Orchestrator Step 1: Initial Data Gathering & Validation]
  Departure: Montréal-Trudeau International Airport (YUL)
  Destination: Shanghai Pudong International Airport (PVG)
  Aircraft: BOEING 777 (Speed: 550 mph, Fuel Rate: 3000 lbs/hr)

[Orchestrator Step 2: User Input Agent - Processing Request]

[Calling LLM as user_input_agent with prompt]: The user wants to plan a flight from YUL to PVG using a BOEING 777. They need a comprehensive plan c...
  User Input Agent processed request: Core Request: The user aims to plan a flight from Montreal (YUL) to Shanghai (PVG) using a Boeing 777 aircraft, considering all necessary factors and assuming normal destination weather.

Initial Key Data Points Needed:

1. Departure date and time for accurate weather forecasts and airport conditions.
2. Detailed flight route, including a